In [1]:
import energyflow
import numpy as np

In [2]:
X, Y = energyflow.datasets.qg_jets.load(num_data=100000, generator='pythia', pad=True, with_bc=False, cache_dir='~/.energyflow')

In [3]:
# preprocess by centering jets and normalizing pts
for x in X:
    mask = x[:,0] > 0
    yphi_avg = np.average(x[mask,1:3], weights=x[mask,0], axis=0)
    x[mask,1:3] -= yphi_avg
    x[mask,0] /= x[:,0].sum()

In [103]:
# preprocess PIDs so they are O(1) or less
X[:,:,3] = X[:,:,3] / 2000

In [ ]:
nparticles = X.shape[1]

In [4]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## Regular NN

In [144]:
inputs = keras.Input(shape=(nparticles,4))
layer = keras.layers.Conv1D(16,1,padding="same",activation="relu")(inputs)
layer = keras.layers.AveragePooling1D(pool_size = nparticles, padding="same")(layer)
layer = keras.layers.Flatten()(layer)
layer = keras.layers.Dense(16,activation="relu")(layer)
output = keras.layers.Dense(1,activation="sigmoid")(layer)
model = keras.Model(inputs=inputs, outputs=output)
model.summary()

model.compile(optimizer=keras.optimizers.Adam(),
             loss = 'binary_crossentropy',metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 139, 4)            0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 139, 16)           80        
_________________________________________________________________
average_pooling1d_32 (Averag (None, 1, 16)             0         
_________________________________________________________________
flatten_29 (Flatten)         (None, 16)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 17        
Total params: 369
Trainable params: 369
Non-trainable params: 0
_________________________________________________________________


In [145]:
history = model.fit(X,Y,epochs=10)

Epoch 1/10
100000/100000 [==============================] - 15s 147us/sample - loss: 0.5624 - acc: 0.7236
Epoch 2/10
100000/100000 [==============================] - 12s 120us/sample - loss: 0.5156 - acc: 0.7507
Epoch 3/10
100000/100000 [==============================] - 13s 128us/sample - loss: 0.4984 - acc: 0.7623
Epoch 4/10
100000/100000 [==============================] - 15s 148us/sample - loss: 0.4866 - acc: 0.7710
Epoch 5/10
100000/100000 [==============================] - 14s 137us/sample - loss: 0.4802 - acc: 0.7750
Epoch 6/10
100000/100000 [==============================] - 12s 124us/sample - loss: 0.4774 - acc: 0.7770
Epoch 7/10
100000/100000 [==============================] - 13s 135us/sample - loss: 0.4754 - acc: 0.7783
Epoch 8/10
100000/100000 [==============================] - 13s 125us/sample - loss: 0.4742 - acc: 0.7806
Epoch 9/10
100000/100000 [==============================] - 11s 114us/sample - loss: 0.4732 - acc: 0.7810
Epoch 10/10
100000/100000 [===================

In [146]:
model.predict(X[:100])

array([[0.9889244 ],
       [0.9857632 ],
       [0.12990975],
       [0.6508647 ],
       [0.7963798 ],
       [0.22441   ],
       [0.9892951 ],
       [0.8941284 ],
       [0.7492796 ],
       [0.98642755],
       [0.9342357 ],
       [0.19584006],
       [0.07013935],
       [0.27346027],
       [0.3341374 ],
       [0.9007114 ],
       [0.200127  ],
       [0.29861665],
       [0.6812169 ],
       [0.27042848],
       [0.06506029],
       [0.9335626 ],
       [0.9340154 ],
       [0.41713715],
       [0.7390985 ],
       [0.40384182],
       [0.66093755],
       [0.43255898],
       [0.4321254 ],
       [0.77069855],
       [0.3253877 ],
       [0.18557853],
       [0.28337917],
       [0.74087775],
       [0.193587  ],
       [0.25558156],
       [0.32523674],
       [0.800222  ],
       [0.88683546],
       [0.24996164],
       [0.3011129 ],
       [0.1338093 ],
       [0.4137681 ],
       [0.9822666 ],
       [0.8969711 ],
       [0.35345945],
       [0.9158151 ],
       [0.890

In [147]:
Y[:100]

array([1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
       0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
       1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1.,
       0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
       0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0.])

## BNN

Fucked it up so far, need to fix.

In [116]:
batch_size = 100

inputs = keras.Input(shape=(nparticles,4))
layer = tfp.layers.Convolution1DFlipout(16,1,padding="same",activation="relu")(inputs)
#layer = tfp.layers.Convolution1DFlipout(16,1,padding="same",activation="relu")(layer)
layer = keras.layers.AveragePooling1D(pool_size = nparticles, padding="same")(layer)
layer = keras.layers.Flatten()(layer)
layer = tfp.layers.DenseFlipout(16,activation="relu")(layer)
output = tfp.layers.DenseFlipout(1,activation="sigmoid")(layer)
model = keras.Model(inputs=inputs, outputs=output)
model.summary()

def custom_loss(y_true,y_pred):

    neg_log_likelihood = keras.losses.binary_crossentropy(
        y_true,
        y_pred
    )
    kl = sum(model.losses)/batch_size
    return neg_log_likelihood + kl

model.compile(optimizer=keras.optimizers.Adam(),
              loss = custom_loss,
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        (None, 139, 4)            0         
_________________________________________________________________
conv1d_flipout_20 (Conv1DFli (None, 139, 16)           144       
_________________________________________________________________
average_pooling1d_22 (Averag (None, 1, 16)             0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 16)                0         
_________________________________________________________________
dense_flipout_26 (DenseFlipo (None, 16)                528       
_________________________________________________________________
dense_flipout_27 (DenseFlipo (None, 1)                 33        
Total params: 705
Trainable params: 705
Non-trainable params: 0
_________________________________________________________________


In [137]:
batch_size = 100

model = keras.Sequential([
    tfp.layers.Convolution1DFlipout(16,1,padding="same",activation="relu",input_shape=(nparticles,4)),
    keras.layers.AveragePooling1D(pool_size = nparticles, padding="same"),
    keras.layers.Flatten(),
    tfp.layers.DenseFlipout(16,activation="relu"),
    tfp.layers.DenseFlipout(1,activation="relu")
])
# model.build(input_shape=(None,nparticles,4))
model.summary()


def custom_loss2(y_true,y_pred):

    neg_log_likelihood = keras.losses.binary_crossentropy(
        y_true,
        y_pred
    )
    kl = sum(model.losses)/10000
    return neg_log_likelihood + kl


logits = model(images)
labels_distribution = tfd.Categorical(logits=logits)

model.compile(optimizer=keras.optimizers.Adam(),
              loss = custom_loss2,
              metrics=['accuracy'])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_flipout_29 (Conv1DFli (None, 139, 16)           144       
_________________________________________________________________
average_pooling1d_31 (Averag (None, 1, 16)             0         
_________________________________________________________________
flatten_28 (Flatten)         (None, 16)                0         
_________________________________________________________________
dense_flipout_44 (DenseFlipo (None, 16)                528       
_________________________________________________________________
dense_flipout_45 (DenseFlipo (None, 1)                 33        
Total params: 705
Trainable params: 705
Non-trainable params: 0
_________________________________________________________________


In [142]:
model(X[:100])

<tf.Tensor 'sequential_8_1/dense_flipout_45/Sigmoid:0' shape=(100, 1) dtype=float32>

In [139]:
history = model.fit(X,Y,epochs=100,batch_size = batch_size)

Epoch 1/100
100000/100000 [==============================] - 9s 94us/sample - loss: 688.9618 - acc: 0.5013
Epoch 2/100
100000/100000 [==============================] - 9s 93us/sample - loss: 389.4827 - acc: 0.4992
Epoch 3/100
100000/100000 [==============================] - 8s 77us/sample - loss: 156.5932 - acc: 0.4970
Epoch 4/100
100000/100000 [==============================] - 8s 82us/sample - loss: 34.2414 - acc: 0.4892
Epoch 5/100
100000/100000 [==============================] - 7s 71us/sample - loss: 3.1962 - acc: 0.5046
Epoch 6/100
100000/100000 [==============================] - 7s 72us/sample - loss: 0.7273 - acc: 0.5119
Epoch 7/100
100000/100000 [==============================] - 8s 78us/sample - loss: 0.6933 - acc: 0.4948
Epoch 8/100
100000/100000 [==============================] - 7s 71us/sample - loss: 0.6933 - acc: 0.4986
Epoch 9/100
100000/100000 [==============================] - 8s 79us/sample - loss: 0.6931 - acc: 0.5051
Epoch 10/100
100000/100000 [====================

 73200/100000 [====================>.........] - ETA: 2s - loss: 0.6932 - acc: 0.4977

KeyboardInterrupt: 

In [123]:
model.predict(X[0:1])

array([[0.49742004]], dtype=float32)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 139, 4)            0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 139, 16)           80        
_________________________________________________________________
average_pooling1d_18 (Averag (None, 1, 16)             0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 16)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 17        
Total params: 369
Trainable params: 369
Non-trainable params: 0
_________________________________________________________________


In [109]:
history = model.fit(X,Y,epochs=100)

Epoch 1/100
100000/100000 [==============================] - 11s 106us/sample - loss: 0.5643 - acc: 0.7218
Epoch 2/100
100000/100000 [==============================] - 13s 127us/sample - loss: 0.5113 - acc: 0.7538
Epoch 3/100
100000/100000 [==============================] - 13s 130us/sample - loss: 0.4937 - acc: 0.7670
Epoch 4/100
100000/100000 [==============================] - 11s 110us/sample - loss: 0.4842 - acc: 0.7736
Epoch 5/100
100000/100000 [==============================] - 14s 141us/sample - loss: 0.4797 - acc: 0.7760
Epoch 6/100
100000/100000 [==============================] - 13s 131us/sample - loss: 0.4775 - acc: 0.7779
Epoch 7/100
100000/100000 [==============================] - 11s 107us/sample - loss: 0.4759 - acc: 0.7793
Epoch 8/100
100000/100000 [==============================] - 13s 128us/sample - loss: 0.4746 - acc: 0.7799
Epoch 9/100
100000/100000 [==============================] - 10s 99us/sample - loss: 0.4735 - acc: 0.7797
Epoch 10/100
100000/100000 [==========

KeyboardInterrupt: 